### Imports

In [13]:
import glob
import pandas as pd
import re
import numpy as np
import nltk

#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### Filter Emoji

In [3]:
def filter_emoji_in_array(datas):
    for i in range(len(datas)):
        datas[i][0] = filter_emoji(datas[i][0])

def filter_emoji(text):
    emoji_pattern = re.compile("["
        u"\U00002700-\U000027BF"  # dingbats
        u"\U00002460-\U0001F2FF"  # enclosed characters
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F910-\U0001F9FF"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U00000080-\U000000FF"  # uncategorized 1 (latin 1 supplement)
        u"\U00002000-\U0000206F"  # uncategorized 2 (general punctuation)
        #u"\U00000000-\U0000007F"  # uncategorized 3 (basic latin)
        u"\U000020D0-\U000020FF"  # uncategorized 4 (Combining Diacritical Marks for Symbols )
        u"\U00002100-\U0000214F"  # uncategorized 5 (Letterlike Symbols )
        u"\U00002300-\U000023FF"  # uncategorized 6 (Misc Technical )
        u"\U000025A0-\U000025FF"  # uncategorized 7 (Geometric Shapes )
        u"\U00002600-\U000026FF"  # uncategorized 8 (Misc Symbols )
        u"\U00002900-\U0000297F"  # uncategorized 9 (Supp Arrows B )
        u"\U00002B00-\U00002BFF"  # uncategorized 10 (Misc Symbol Arrows )
        u"\U00003000-\U0000303F"  # uncategorized 11 (CJK Sumbols and Punc )
        u"\U00003200-\U000032FF"  # uncategorized 12 (Enclosed CJK Letters and Months)
        u"\U0001F000-\U0001F02F"  # uncategorized 13 (Mahjong Tiles)
        u"\U0001F0A0-\U0001F0FF"  # uncategorized 14 (Playing Cards)
        u"\U0001F300-\U0001F5FF"  # uncategorized 15 (Misc Symbols and Pict)
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return(emoji_pattern.sub(r'', text))


### Lowercase

In [4]:
def lowercase_array(datas):
    result = datas
    for i in range(len(datas)):
        result[i][0] = datas[i][0].lower()
    datas = result

In [49]:
def apply_first_normalization(datas):
    filter_emoji_in_array(datas)
    lowercase_array(datas)

### Read all csv

In [98]:
def is_contain(check_data, datas):
    i = 0
    while ((i < len(datas)) and (check_data[0] != datas[i][0])) :
        i = i + 1
    return (i < len(datas))

def filter_same_data(check_datas, datas):
    new_array = datas
    for i in range(len(check_datas)) :
        if (not(is_contain(check_datas[i], new_array))):
            if (len(new_array) == 0):
                new_array = [check_datas[i]]
            else :
                new_array = np.append(new_array,[check_datas[i]], axis = 0)
    new_array = pd.DataFrame(data = new_array).values
    return (new_array)

def read_all_csv():
    result = []
    attribute_names = ["","tweet","user"]
    csvs_path = glob.glob('raw_data/*.csv')
    for path in csvs_path :
        data_input = pd.read_csv(path, header=0, usecols=range(1,3))
        data_frame_input = pd.DataFrame(data = data_input).values
        apply_first_normalization(data_frame_input)
        result = filter_same_data(data_frame_input, result)
    data_frame_result = pd.DataFrame(data = result)
    return (data_frame_result.values)

raw_data = read_all_csv()
raw_data = pd.DataFrame(data=raw_data)

print(len(raw_data))
print(raw_data)

1344
                                                      0                1
0      ya anjir cpt bgt ya abis barang manusia diskon:(         SOHEEDCH
1     lazada 11.11.2019 diskon terbesar 24 jam https...        Vallno_07
2     selama kalo liat promo diskon 70% + 30% dalem ...         bayu_joo
3     apa bedanya matahari sama bulan?  matahari ada...       Hanifati_f
4     @garudacares bagaimana cara tukar mileage disk...           Nug_QA
5     @gencamon nahiya benerrr hahahaha kalo makan c...         primetam
6            fak gmarket kenapa diskon nya sekarang sih       singularyv
7     sejarah asia tenggara karya m.c. ricklefs rp 3...   KomunitasBambu
8     @amingcoffee transmart kuburaya diskon, kapan ...      kalbarinfo_
9     tah diskon 50% cenah..  yuk diserbu..   https:...   AmaAbiyyunShoh
10    @wongalas901 @prabowo @aniesbaswedan rika wis ...     Dianhandoko6
11    sebenarnya bahagia itu sederhana... - beli ini...        anggieppu
12    yuhuu..  siapa lagi yang mau order deodo

### Save

In [7]:
def save_as_csv(dataframe) :
    dataframe.to_csv("integrated_data/integrated.csv", encoding='utf-8')

save_as_csv(raw_data)

### Filter Non Words

In [106]:
def filter_url(text):
    return(re.sub(r"http\S+", "", text))

def filter_tag(text):
    return(re.sub(r"@\S+", "", text))

def filter_hashtag(text):
    return(re.sub(r"#\S+", "", text))

def filter_not_alphanumeric(text):
    return(re.sub(r'[^\w]', ' ', text))

def filter_number(text):
    return(re.sub(r'[0-9]', '', text))
    
print(raw_data[0][445])
print(filter_url(raw_data[0][445]))

print(raw_data[0][445])
print(filter_tag(raw_data[0][445]))

print(raw_data[0][1337])
print(filter_hashtag(raw_data[0][1337]))

print(raw_data[0][0])
print(filter_not_alphanumeric(raw_data[0][0]))

diskon dapat di duga! ga nyesel deh milih markobar wkwk @kaesangp https://t.co/dxmmbi1nc1
diskon dapat di duga! ga nyesel deh milih markobar wkwk @kaesangp 
diskon dapat di duga! ga nyesel deh milih markobar wkwk @kaesangp https://t.co/dxmmbi1nc1
diskon dapat di duga! ga nyesel deh milih markobar wkwk  https://t.co/dxmmbi1nc1
road to fantastic 11! rayainnya #belanjabarengtemen  beli 3 produk ekstra diskon 25%!  *) hanya berlaku di halaman ini *) akan terpotong pada saat proses check out   __  yuk belanja! https://t.co/c9nhfvk5gp  pic: pinterest  #sabtu #diskon #infojakarta #infopromo  #freeongkir https://t.co/ifgzxb7cuz
road to fantastic 11! rayainnya   beli 3 produk ekstra diskon 25%!  *) hanya berlaku di halaman ini *) akan terpotong pada saat proses check out   __  yuk belanja! https://t.co/c9nhfvk5gp  pic: pinterest        https://t.co/ifgzxb7cuz
ya anjir cpt bgt ya abis barang manusia diskon:(
ya anjir cpt bgt ya abis barang manusia diskon  


In [107]:
def filter_non_words(datas):
    result = datas.copy(deep=True)
    for i in range(len(datas)):
        result[0][i] = filter_url(result[0][i])
        result[0][i] = filter_tag(result[0][i])
        result[0][i] = filter_hashtag(result[0][i])
        result[0][i] = filter_not_alphanumeric(result[0][i])
        result[0][i] = filter_number(result[0][i])
    return result

only_words_datas = filter_non_words(raw_data)
print(only_words_datas)

                                                      0                1
0      ya anjir cpt bgt ya abis barang manusia diskon           SOHEEDCH
1                       lazada    diskon terbesar  jam         Vallno_07
2     selama kalo liat promo diskon       dalem hati...         bayu_joo
3     apa bedanya matahari sama bulan   matahari ada...       Hanifati_f
4      bagaimana cara tukar mileage diskon    garuda...           Nug_QA
5      nahiya benerrr hahahaha kalo makan cantik ken...         primetam
6            fak gmarket kenapa diskon nya sekarang sih       singularyv
7     sejarah asia tenggara karya m c  ricklefs rp  ...   KomunitasBambu
8      transmart kuburaya diskon  kapan lagi rp   ng...      kalbarinfo_
9                tah diskon   cenah    yuk diserbu        AmaAbiyyunShoh
10       rika wis inden mobil esemka nek arp tuku ga...     Dianhandoko6
11    sebenarnya bahagia itu sederhana      beli ini...        anggieppu
12    yuhuu    siapa lagi yang mau order deodorantn

### Split Words

In [108]:
def words_list_array(datas):
    result = []
    for i in range(len(datas)):
        new_datas = nltk.word_tokenize(datas[0][i])
        new_datas = pd.DataFrame(data=new_datas).values
        result = filter_same_data(new_datas, result)
        #result.extend(new_datas)
    result = pd.DataFrame(data=result)
    return result

def words_list(data):
    return(nltk.word_tokenize(data))

word_list_datas = words_list_array(only_words_datas)
print(len(word_list_datas))
print(word_list_datas)

5027
                 0
0               ya
1            anjir
2              cpt
3              bgt
4             abis
5           barang
6          manusia
7           diskon
8           lazada
9         terbesar
10             jam
11          selama
12            kalo
13            liat
14           promo
15           dalem
16            hati
17        langsung
18          bilang
19             ini
20            jadi
21          gratis
22            dong
23             itu
24        tandanya
25            anda
26           masih
27          miskin
28             apa
29         bedanya
...            ...
4997           brt
4998       koridor
4999           iii
5000   menyarankan
5001      nunjukin
5002       sekolah
5003      belinyaa
5004      dimaanaa
5005           opp
5006          road
5007     rayainnya
5008     terpotong
5009        proses
5010            sg
5011           feb
5012  wekawekaweka
5013         laahh
5014          olih
5015         namba
5016      nganyang
5017   